In [7]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

answer = chat.predict("The number of planets")

answer

'in our solar system is eight.'

In [8]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a frontend expert. Also, you only reply in Korean"),
    AIMessage(content="안녕하세요"),
    HumanMessage(content="What is the advantage of Next.js? Also, What's your name?"),
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! Next.js의 장점은 다음과 같습니다:\n\n1. 서버 사이드 렌더링 (Server Side Rendering, SSR): Next.js는 기본적으로 SSR을 지원하여 초기 로딩 속도를 향상시킵니다. 이로 인해 사용자들은 빠른 페이지 로딩 및 검색 엔진 최적화를 경험할 수 있습니다.\n2. 정적 사이트 생성 (Static Site Generation, SSG): Next.js는 정적 사이트 생성을 지원하여 사전 렌더링을 통해 페이지를 생성할 수 있습니다. 이는 동적 데이터가 필요하지 않은 경우에 효과적인 성능을 제공합니다.\n3. 개발 생산성: Next.js는 선호하는 개발 스택과 함께 사용할 수 있으며, 자동 라우팅, Hot Module Replacement 등의 개발 편의 기능을 제공하여 개발 생산성을 높여줍니다.\n4. 확장성: Next.js는 모듈화된 구조를 가지고 있어 작은 규모의 프로젝트부터 대규모의 애플리케이션까지 다양한 규모의 프로젝트에 적합합니다.\n\n그리고 제 이름은 AI 기반 가상어시스턴트입니다. 도움이 필요하신 점이 있으면 언제든지 말씀해주세요!')